In [4]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random
import matplotlib.pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

import GameManager
from GameManager import *

import Neat_AI
from Neat_AI import *

import AI_modules
from AI_modules import *

import my_reporters
from my_reporters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config-feedforward')

In [3]:
dimensions = (15,15)
units_per_side = 11
manager = map_manager(dimensions)

# for i in range(games_run):
#also resets map
manager.setup_layouts_rand(10, units_per_side)
manager.apply_map_layout(1)
print(manager)

manager.curr_team = 1
while (manager.game_joever() == -1 and manager.turn_count < 15): #Turn Count limit may have to be modified
    for unit in manager.Teams[manager.curr_team].live_units:
        win_move = (0, 0)
        if manager.curr_team == 0:
            win_move = neat_ai(manager, unit, best_networks[3])
            #win_move = script_ai(manager, unit)
        elif manager.curr_team == 1:
            win_move = script_ai(manager, unit)
            #win_move = neat_ai(manager, unit, best_networks[2])
        manager.move_unit(unit, win_move)
    print(manager)
    manager.Turn()
#print(manager.game_feedback())

[ ][ ][c][ ][ ][ ][ ][ ][ ][ ][ ][ ][C][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][c][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][C]
[ ][ ][a][ ][ ][ ][ ][ ][ ][ ][ ][ ][A][ ][ ]
[a][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][A][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][I][ ][ ]
[ ][i][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][i][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][I][ ][ ]
[i][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][I][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[i][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][I][ ][ ]
[a][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][A][ ]
[ ][a][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][A][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[c][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][C][ ]

(0, 8)
(2, 10)
(3, 10)
(4, 12)
(5, 10)
(7, 10)
(8, 10)
(10, 10)
(10, 12)
(10, 13)
(13, 12)
[ ][ ][c][ ][ ][ ][ ][ ][C][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][c][ ][ ][ ][ ][ ][ ][ ][C][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ][ ][ ][ ][A][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][

NameError: name 'best_networks' is not defined

In [12]:
print(best_networks)
#best_networks_copy = copy.deepcopy(best_networks)

[<neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A9978580>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A98A2E80>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A7C75880>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A9A26070>]


In [8]:
print(best_networks)
best_networks_copy2 = copy.deepcopy(best_networks)

[<neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A9978580>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A98A2E80>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A7C75880>, <neat.nn.feed_forward.FeedForwardNetwork object at 0x00000297A9A26070>]


In [ ]:
random.seed(1648) #This is to set the random generated board starting positions
best_networks = []
eval_rep = []
win_net, win_genome, stats = Neat_AI.run(config_path, best_networks, eval_rep, "Expanded Game Updated, Ensemble")


 ****** Running generation 0 ****** 

Population's average fitness: 0.27280 stdev: 0.14520
Best fitness: 0.70000 - size: (2, 337) - species 1 - id 41
Best genome Winrate vs. script : 0.04
Best genome mean_dist vs. script : 7.02
Best genome prop_f_all vs. script : 0.01
Best genome prop_h_all vs. script : 0.04
Best genome prop_f_i vs. script : 0.00
Best genome prop_h_i vs. script : 0.01
Best genome prop_f_a vs. script : 0.00
Best genome prop_h_a vs. script : 0.13
Best genome prop_f_c vs. script : 0.03
Best genome prop_h_c vs. script : 0.01
Best genome mean_loc_all vs. script : (6.20, 1.81)
Best genome mean_loc_i vs. script : (8.26, 2.26)
Best genome mean_loc_a vs. script : (3.98, 1.15)
Best genome mean_loc_c vs. script : (8.53, 1.55)
###############################################
Average adjusted fitness: 0.212
Mean genetic distance 2.089, standard deviation 0.597
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====


Best genome Winrate vs. script : 0.05
Best genome mean_dist vs. script : 7.86
Best genome prop_f_all vs. script : 0.31
Best genome prop_h_all vs. script : 0.06
Best genome prop_f_i vs. script : 0.00
Best genome prop_h_i vs. script : 0.23
Best genome prop_f_a vs. script : 0.00
Best genome prop_h_a vs. script : 0.00
Best genome prop_f_c vs. script : 0.54
Best genome prop_h_c vs. script : 0.05
Best genome mean_loc_all vs. script : (4.64, 3.87)
Best genome mean_loc_i vs. script : (11.21, 2.54)
Best genome mean_loc_a vs. script : (3.80, 1.03)
Best genome mean_loc_c vs. script : (3.22, 4.41)
###############################################
Average adjusted fitness: 0.300
Mean genetic distance 1.275, standard deviation 0.362
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    7    23      0.7    0.268     5
     2    7    27      0.9    0.332     1
Total extinctions: 0
Generation time: 138.199 sec (145.579 average

Population's average fitness: 0.64080 stdev: 0.14713
Best fitness: 0.88000 - size: (4, 136) - species 1 - id 684
Best genome Winrate vs. script : 0.26
Best genome mean_dist vs. script : 6.60
Best genome prop_f_all vs. script : 0.03
Best genome prop_h_all vs. script : 0.04
Best genome prop_f_i vs. script : 0.00
Best genome prop_h_i vs. script : 0.00
Best genome prop_f_a vs. script : 0.00
Best genome prop_h_a vs. script : 0.05
Best genome prop_f_c vs. script : 0.08
Best genome prop_h_c vs. script : 0.12
Best genome mean_loc_all vs. script : (6.66, 0.94)
Best genome mean_loc_i vs. script : (6.45, 0.46)
Best genome mean_loc_a vs. script : (8.91, 0.44)
Best genome mean_loc_c vs. script : (5.66, 2.20)
###############################################
Average adjusted fitness: 0.390
Mean genetic distance 1.110, standard deviation 0.299
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   15    31      0.9    0.341   

Population's average fitness: 0.59920 stdev: 0.19976
Best fitness: 0.94000 - size: (3, 133) - species 2 - id 1033
Best genome Winrate vs. script : 0.08
Best genome mean_dist vs. script : 6.82
Best genome prop_f_all vs. script : 0.26
Best genome prop_h_all vs. script : 0.08
Best genome prop_f_i vs. script : 0.00
Best genome prop_h_i vs. script : 0.19
Best genome prop_f_a vs. script : 0.00
Best genome prop_h_a vs. script : 0.07
Best genome prop_f_c vs. script : 0.52
Best genome prop_h_c vs. script : 0.02
Best genome mean_loc_all vs. script : (4.85, 1.38)
Best genome mean_loc_i vs. script : (7.17, 0.92)
Best genome mean_loc_a vs. script : (2.31, 0.40)
Best genome mean_loc_c vs. script : (5.79, 2.18)
###############################################
Average adjusted fitness: 0.438
Mean genetic distance 1.183, standard deviation 0.516
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   23    23      0.9    0.416  

In [30]:
print(eval_rep[0].eval_stats['script'])

[{'mean_dist': 7.063476290267512, 'prop_f_all': 0.07880958984978391, 'prop_h_all': 0.08139674701850187, 'prop_f_i': 0.0, 'prop_h_i': 0.26227690812531473, 'prop_f_a': 0.056598266499582287, 'prop_h_a': 0.016486920426065164, 'prop_f_c': 0.2020856026423555, 'prop_h_c': 0.0, 'mean_loc_all': (3.9870243207156846, 3.7111470667730444), 'mean_loc_i': (5.263299465540093, 0.8518073957271696), 'mean_loc_a': (2.887160521062902, 0.8290483662257075), 'mean_loc_c': (5.9992049262200124, 5.000692954807538)}, {'mean_dist': 7.063476290267512, 'prop_f_all': 0.07880958984978391, 'prop_h_all': 0.08139674701850187, 'prop_f_i': 0.0, 'prop_h_i': 0.26227690812531473, 'prop_f_a': 0.056598266499582287, 'prop_h_a': 0.016486920426065164, 'prop_f_c': 0.2020856026423555, 'prop_h_c': 0.0, 'mean_loc_all': (3.9870243207156846, 3.7111470667730444), 'mean_loc_i': (5.263299465540093, 0.8518073957271696), 'mean_loc_a': (2.887160521062902, 0.8290483662257075), 'mean_loc_c': (5.9992049262200124, 5.000692954807538)}, {'mean_dist

In [4]:
for i in range(len(best_networks)):
    network = best_networks[i]
    with open('./best/networks/{}-network'.format(i), "wb") as f:
        pickle.dump(network, f)
        f.close()

In [234]:
win_net1 = win_net
stats1 = stats

In [9]:
dimensions = (8,8)
units_per_side = 5
manager = map_manager(dimensions)

# for i in range(games_run):
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
script_performance(manager, win_net, config)

NameError: name 'win_net' is not defined

In [267]:
#Restore a checkpoint to resume Run
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
a
p = neat.checkpoint.Checkpointer.restore_checkpoint("./checkpoints/Global-Position-Eval-Func_vsScript_difference/neat-checkpoint-29")
winner = p.run(eval_genomes, 1)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

Mean genetic distance 1.323, standard deviation 0.442


In [172]:
#Restoring a best network for testing
net = load_net("./best/Relative-Position-Eval-Func-config", "./best/Relative-Position-Eval-Func-genome")
print(net)